Il faut chopper date début, fin , longeur 

Il faudrait rajouter : 

    -str deviation 
    -min 
    -max
    -30 dernières minutes 
    -etc...
    
    Mais garder en tête que que ça augmente considérablement la taille de nos features 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

from transplant.tools.dataset import Dataset
dataset=Dataset()

Ici on choppe le début , la fin et la longeure de l'opération 

In [ ]:

        from transplant.tools.learningset import Learningset
        learningset = Learningset()
        train_static_0, test_static_0 = learningset.get_static_filled()

        train_dynamic_0, test_dynamic_0 = dataset.get_dynamic()

        mean_dynamic_train = train_dynamic_0.groupby(['id_patient']).mean().mean()
        
        train_dynamic_0 = train_dynamic_0.fillna(mean_dynamic_train)
        test_dynamic_0 = test_dynamic_0.fillna(mean_dynamic_train)

        def add_start_end_length_op_to_static(X_stat, X_dyn):
            grouped_time = X_dyn.groupby(['id_patient'])['time']

            time_start_df = grouped_time.first().to_frame()
            time_start_df.columns = ['start_operation']
            time_start_df['id_patient'] = time_start_df.index

            X_return = pd.merge(X_stat, time_start_df, on='id_patient')

            time_ends_df = grouped_time.last().to_frame()
            time_ends_df.columns = ['ends_operation']
            time_ends_df['id_patient'] = time_ends_df.index

            X_return = pd.merge(X_return, time_ends_df, on='id_patient')

            X_return['length_op'] = X_return['ends_operation'] - X_return['start_operation']

            return X_return

        train_static_1 = add_start_end_length_op_to_static(train_static_0, train_dynamic_0)
        test_static_1 = add_start_end_length_op_to_static(test_static_0, test_dynamic_0)


In [ ]:
from transplant.tools.learningset import Learningset

learningset = Learningset()

train_static,test_static = learningset.get_static_filled()

#Dynamic (flatten)
train_dynamic_0, test_dynamic_0 = dataset.get_dynamic()



train_dynamic = train_dynamic_0.fillna(0) #Ici contrairement à avant on remplace par des zéro 
test_dynamic = test_dynamic_0.fillna(0)

# Merging
def merge_dyn_sta(X_train_static, X_train_dynamic, X_test_static, X_test_dynamic):
    return pd.merge(X_train_static, X_train_dynamic, on='id_patient'), pd.merge(X_test_static, X_test_dynamic, on='id_patient')


In [ ]:
liste_func=[np.mean,np.std,np.median,max,min]
liste_func_name=['mean','std','median','max','min']

Avoir le mean

In [ ]:
train_glob_0, test_glob_0 = train_static , test_static

for i in range(len(liste_func)) :
    print('On en est à '+liste_func_name[i])
    func=liste_func[i]
    
    train_grouped=train_dynamic.groupby(['id_patient'], as_index=False) #On enregistre les groupes
    test_grouped=test_dynamic.groupby(['id_patient'], as_index=False)
    
    train_id=train_grouped['id_patient'].apply(np.mean)  #On conserve le id du patient
    test_id=test_grouped['id_patient'].apply(np.mean)
    
   
    train_dynamic_flat = train_grouped.apply(func)   #On applique la fonction
    test_dynamic_flat = test_grouped.apply(func)

    train_dynamic_flat.rename(columns=lambda x: x+'_'+liste_func_name[i] if x!='id_patient' else x, inplace=True)
    test_dynamic_flat.rename(columns=lambda x: x+'_'+liste_func_name[i]  if x!='id_patient' else x, inplace=True)
    
    train_dynamic_flat['id_patient']=train_id
    test_dynamic_flat['id_patient']=test_id
    
    train_glob_0, test_glob_0 = merge_dyn_sta(train_glob_0, train_dynamic_flat, test_glob_0, test_dynamic_flat)
    


In [ ]:
train_glob_0.shape

In [ ]:
print([i for i in train_glob_0.columns])

In [ ]:
train_glob_0

In [ ]:
train_dynamic_flat.columns

In [ ]:
def center_reduce_data(W_train, W_test):
    mean_train = W_train.mean()
    std_train = W_test.std()

    return (W_train-mean_train)/std_train, (W_test-mean_train)/std_train

dic_to_One_Hot = {0: [1, 0], 1: [0, 1]}

y_train_cls = np.array(train_glob['target'])
y_train_hot = np.array(list(train_glob['target'].map(dic_to_One_Hot)))

y_test_cls = np.array(test_glob['target'])
y_test_hot = np.array(list(test_glob['target'].map(dic_to_One_Hot)))

if centered_reduced:
    X_train, X_test = center_reduce_data(train_glob.drop(['target'], axis=1), test_glob.drop(['target'], axis=1))

    X_train = np.array(X_train)
    X_test = np.array(X_test)

else:
    X_train = np.array(train_glob.drop(['target'], axis=1))
    X_test = np.array(test_glob.drop(['target'], axis=1))

        # Return
if target_format == "cls":
    return X_train, X_test, y_train_cls, y_test_cls, train_glob.drop(['target'], axis=1).columns

if target_format == "One_Hot":
    return X_train, X_test, y_train_hot, y_test_hot, train_glob.drop(['target'], axis=1).columns
